In [68]:
from json_extractor import from_file_get_n_docs
from token_list_mapper import map_token_list_a_to_b
from similarity import similar

# Get Parsed docs

In [4]:
STANFORD_FILENAME = '100_parsed_from_stanford.jsonl'
TAGME_FILENAME    = '100_parsed_from_tagme.jsonl'
WIKIFIER_FILENAME = '100_parsed_from_wikifier.jsonl'

In [5]:
stanford_parsed_docs = from_file_get_n_docs(STANFORD_FILENAME, 5)

In [6]:
wikifier_parsed_docs = from_file_get_n_docs(WIKIFIER_FILENAME, 5)

In [12]:
tagme_parsed_docs    = from_file_get_n_docs(TAGME_FILENAME, 5)

In [13]:
test_index = 0

In [15]:
stanford_doc = stanford_parsed_docs[test_index]

In [16]:
wikifier_doc = wikifier_parsed_docs[test_index]

In [49]:
tagme_doc = tagme_parsed_docs[test_index]

# Generate Word Dicts

## Common token list + tagged-words dictionary => tagged-words list

In [56]:
filter_nones = lambda p: [v for v in enumerate(p) if v[1] is not None]

In [55]:
def map_parsed_doc_to_token_list(doc, token_list):
    token_list_to_doc = map_token_list_a_to_b(token_list, doc['tokens'])
    parsed_token_list = map(
        lambda (i, _): doc['tagged-words'].get(str(token_list_to_doc.get(i, -1)), None),
        enumerate(token_list)
    )
    return parsed_token_list
        
# filter_nones(map_parsed_doc_to_token_list(stanford_doc, stanford_doc['tokens'])).__len__(), stanford_doc['tagged-words'].keys().__len__()
# filter_nones(map_parsed_doc_to_token_list(tagme_doc, stanford_doc['tokens'])).__len__(), tagme_doc['tagged-words'].keys().__len__()
# map(lambda tag: (tag[1], stanford_doc['tokens'][tag[0]]), filter_nones(map_parsed_doc_to_token_list(tagme_doc, stanford_doc['tokens'])))

In [59]:
tagme_mapped = map_parsed_doc_to_token_list(tagme_doc, stanford_doc['tokens'])
filter_nones(tagme_mapped).__len__(), tagme_doc['tagged-words'].keys().__len__()

(34, 137)

In [61]:
tagme_doc['tagged-words']

{u'0': {u'dbpedia_categories': [u'Military personnel',
   u"Military veterans' affairs",
   u'Aftermath of war'],
  u'end': 0,
  u'id': 327806,
  u'link_probability': u'0.01097',
  u'rho': u'0.04836',
  u'spot': u'VETERANS',
  u'start': 0,
  u'title': u'Veteran'},
 u'1': {u'dbpedia_categories': [u'Military life',
   u'Hand gestures',
   u'Greetings'],
  u'end': 1,
  u'id': 28977,
  u'link_probability': u'0.00269',
  u'rho': u'0.03156',
  u'spot': u'saluted',
  u'start': 1,
  u'title': u'Salute'},
 u'10': {u'dbpedia_categories': [u'Metropolitan Police',
   u'Police forces of London',
   u'Organizations established in 1829',
   u'1829 establishments in England'],
  u'end': 10,
  u'id': 192450,
  u'link_probability': u'0.00127',
  u'rho': u'0.05313',
  u'spot': u'met',
  u'start': 10,
  u'title': u'Metropolitan Police Service'},
 u'100': {u'dbpedia_categories': [u'Group processes'],
  u'end': 101,
  u'id': 648520,
  u'link_probability': u'0.01211',
  u'rho': u'0.01286',
  u'spot': u'one u

In [83]:
similar_terms = [(similar(w, u'Germany', None), w) for (i,w) in enumerate(stanford_doc['tokens']) if similar (w ,u'Germany', 0.7)]
similar_terms

[(0.9333333333333333, u'Germany.'), (0.7272727272727273, u'many')]

In [87]:
not not [1]

True